https://debuggercafe.com/a-guide-to-xgboost-in-python/


**Импортируем необходимые библиотеки**

In [6]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

import xgboost as xgb
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


**Загрузка CSV-файла с компьютера**

In [77]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

import io
data = pd.read_csv(io.StringIO(uploaded['titanic.csv'].decode('utf-8')))

KeyError: ignored

**Загрузка CSV-файла с гугл диска**

In [78]:
from google.colab import drive
drive.mount('/content/drive')

data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data_19-20.csv', encoding = 'utf-8')

data = pd.DataFrame(data)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,1,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
#data.head()
data

**Обработка**

In [ ]:
# Проверка категориальности
print(data.dtypes)

In [79]:
# Убираем ненужные признаки
data = data.drop(['Код_ученика'], axis=1)
print(data.dtypes)

Класс           object
Код_предмета    object
ЛогинОО         object
Балл            object
Отметка         object
Year             int64
dtype: object


In [80]:
# Проверка пропущенных значений
print(data.isna().sum())

Класс           0
Код_предмета    0
ЛогинОО         0
Балл            0
Отметка         0
Year            0
dtype: int64


In [ ]:
# Если есть пропущенные значения
data['Age'].fillna(data['Age'].median(), inplace=True)
data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)

In [81]:
data

,Класс,Код_предмета,ЛогинОО,Балл,Отметка,Year
0,4,2,sch01110001,11,4,2019
1,4,2,sch01110001,18,5,2019
2,4,2,sch01110001,9,3,2019
3,4,2,sch01110001,11,4,2019
4,4,2,sch01110001,8,3,2019
...,...,...,...,...,...,...
11221344,5,1,sch87176023,10,2,2020
11221345,5,1,sch87176023,21,3,2020
11221346,5,1,sch87176023,21,3,2020
11221347,5,1,sch87176023,21,3,2020


In [ ]:
data['Класс']

In [82]:
for col in ('Класс', 'Код_предмета', 'Балл', 'Отметка'):
  data[col] = data[col].str.replace(' ', '')
  #data[col] = data[col].str.replace('\t', '')
  #data[col] = data[col].str.replace('\n', '')


In [83]:
data

,Класс,Код_предмета,ЛогинОО,Балл,Отметка,Year
0,NaN,NaN,sch01110001,NaN,NaN,2019
1,NaN,NaN,sch01110001,NaN,NaN,2019
2,NaN,NaN,sch01110001,NaN,NaN,2019
3,NaN,NaN,sch01110001,NaN,NaN,2019
4,NaN,NaN,sch01110001,NaN,NaN,2019
...,...,...,...,...,...,...
11221344,NaN,NaN,sch87176023,NaN,NaN,2020
11221345,NaN,NaN,sch87176023,NaN,NaN,2020
11221346,NaN,NaN,sch87176023,NaN,NaN,2020
11221347,NaN,NaN,sch87176023,NaN,NaN,2020


In [ ]:
# Кодирование
encoder = LabelEncoder()

data['ЛогинОО'] = encoder.fit_transform(data['ЛогинОО'])

for col in ('Класс', 'Код_предмета', 'Балл', 'Отметка'):

  data[col] = pd.to_numeric(data[col], errors='raise')


  #data[col] = data[col].str.replace(' ', '')
  #data[col] = data[col].astype(int)

print(data.dtypes)
data

**Разделение наборов**

In [ ]:
# Разделение параметров и меток
features = data.drop(['Балл', 'Отметка'], axis=1)
labels_1 = data['Балл']
labels_2 = data['Отметка']

# train-test (85% - обучение, 15% - валидация)
x_train_1, x_val_1, y_train_1, y_val_1 = train_test_split(features, labels_1, train_size=0.85)
x_train_2, x_val_2, y_train_2, y_val_2 = train_test_split(features, labels_2, train_size=0.85)

In [ ]:
# Преобразование в DMatrix
d_train_1 = xgb.DMatrix(x_train_1, y_train_1)
d_test_1 = xgb.DMatrix(x_val_1, y_val_1)

d_train_2 = xgb.DMatrix(x_train_2, y_train_2)
d_test_2 = xgb.DMatrix(x_val_2, y_val_2)

**Настройка параметров**

*booster* указывает тип бустера, который будет использовать XGBoost. По умолчанию gbtree.

*max_depth* - максимальная глубина дерева. Очень высокое значение может привести к перегрузке алгоритма.

*learning_rate* - скорость обучения алгоритма, которая установлена на 0.1. Указание, как быстро наш алгоритм будет сходиться к оптимальному решению.

*sample_type* - тип выборки, используемый для отброшенных деревьев. Может быть либо *uniform*, либо *weighted*.

*normalize_type* - тип алгоритма нормализации, который будет использоваться во время обучения.

*objective* - цель обучения для алгоритма. *binary:hinge* выдает результаты классификации (либо 0, либо 1).

*rate_drop* - коэффициент отсева деревьев. Диапазон от 0.0 до 1.0. Использовали 0.1, значит 10% деревьев будут отброшены.

*n_estimators* -  количество усиленных деревьев для использования во время обучения.

In [ ]:
params_0 = {
    'tree_method': 'approx', # 'auto' не подходит, слижком большой датасет
    'booster': 'gbtree',
    'max_depth': 5, 
    'learning_rate': 0.1,
    'sample_type': 'uniform',
    'normalize_type': 'tree',
    'objective': 'reg:squarederror', # регрессия с квадратом потерь
    'rate_drop': 0.1,
    'n_estimators': 500
}

In [ ]:
xgb_clf_1 = xgb.train(params_0, d_train_1)
xgb_clf_2 = xgb.train(params_0, d_train_2)

**Визуализация важности параметров**

In [ ]:
plt.figure(figsize=(16, 12))
xgb.plot_importance(xgb_clf_1)
plt.show()

plt.figure(figsize=(16, 12))
xgb.plot_importance(xgb_clf_2)
plt.show()

In [ ]:
# Прогноз
preds_1 = xgb_clf_1.predict(d_test_1)
preds_2 = xgb_clf_2.predict(d_test_2)
#print(preds)

# Оценка точности
print(np.round(accuracy_score(y_val_1, preds_1)*100, 2), '%')
print(np.round(accuracy_score(y_val_2, preds_2)*100, 2), '%')

**Кросс-валидация k-fold**

In [ ]:
# DMatrix, состоящая из всех обучающих данных
dmatrix_data_1 = xgb.DMatrix(features, labels_1)
dmatrix_data_2 = xgb.DMatrix(features, labels_2)

In [ ]:
# Параметры
params = {
    'objective': 'reg:squarederror', # регрессия с квадратом потерь
    'colsample_bytree': 0.3,
    'learning_rate': 0.1,
    'max_depth': 5,
}

Параметры xgb.cv():

*num_boost_round* - количество итераций бустинга, для которых выполняется перекрестная проверка.

*early_stopping_rounds* - если метрика валидации не улучшится за указанные раунды (в нашем случае 10), то кросс-валидация прекратится.

*metrics* - метрика, по которой оценивается перекрестная проверка. *error* метрика обычно используется для двоичной классификации.

*as_pandas* - результат в виде фрейма данных pandas.

In [ ]:
# Кросс-валидация
cross_val_1 = xgb.cv(
    params=params,
    dtrain=dmatrix_data_1, 
    nfold=3,
    num_boost_round=50,
    early_stopping_rounds=10,
    metrics='error',
    as_pandas=True,
    seed=42)
cross_val_2 = xgb.cv(
    params=params,
    dtrain=dmatrix_data_2, 
    nfold=3,
    num_boost_round=50,
    early_stopping_rounds=10,
    metrics='error',
    as_pandas=True,
    seed=42)

print(cross_val_1.head())
print(cross_val_2.head())